In [ ]:
import time

notebook_start_time = time.time()

# Set up environment

In [ ]:
import sys
from pathlib import Path


def is_google_colab() -> bool:
    if "google.colab" in str(get_ipython()):
        return True
    return False


def clone_repository() -> None:
    !git clone https://github.com/decodingml/hands-on-recommender-system.git
    %cd hands-on-recommender-system/


def install_dependencies() -> None:
    !pip install --upgrade uv
    !uv pip install --all-extras --system --requirement pyproject.toml


if is_google_colab():
    clone_repository()
    install_dependencies()

    root_dir = str(Path().absolute())
    print("⛳️ Google Colab environment")
else:
    root_dir = str(Path().absolute().parent)
    print("⛳️ Local environment")

# Add the root directory to the `PYTHONPATH` to use the `recsys` Python module from the notebook.
if root_dir not in sys.path:
    print(f"Adding the following directory to the PYTHONPATH: {root_dir}")
    sys.path.append(root_dir)

# Inference pipeline: Deploying and testing the LLM ranker inference pipeline

In this notebook, we will dig into the inference pipeline and deploy it to Hopsworks as a real-time service.

## 📝 Imports

In [ ]:
import warnings

warnings.filterwarnings("ignore")

from loguru import logger

from recsys import hopsworks_integration

## <span style="color:#ff5f27">🔮 Connect to Hopsworks Feature Store </span>

In [ ]:
project, fs = hopsworks_integration.get_feature_store()

# Deploying the ranking inference pipeline


First you need to register your LLM ranking model:

In [ ]:
ranking_model = hopsworks_integration.llm_ranking_serving.HopsworksLLMRankingModel()
ranking_model.register(project.get_model_registry())

Then you can deploy your LLM ranking model, which implements a `Predict` class that tells Hopsworks how to perform inference on it:

In [ ]:
ranking_deployment = hopsworks_integration.llm_ranking_serving.HopsworksLLMRankingModel.deploy()

Now, we have to explicitly start the deployment:

In [ ]:
ranking_deployment.start()

## <span style="color:#ff5f27"> Test the ranking inference pipeline</span>


In [ ]:
def get_top_recommendations(ranked_candidates, k=3):
    return [candidate[-1] for candidate in ranked_candidates["ranking"][:k]]

Let's define a dummy test example to test our ranking deployment (only the `customer_id` has to match):

In [ ]:
test_ranking_input = [
        {
            "customer_id": "d327d0ad9e30085a436933dfbb7f77cf42e38447993a078ed35d93e3fd350ecf",
            "month_sin": 1.2246467991473532e-16,
            "query_emb": [
                0.214135289,
                0.571055949,
                0.330709577,
                -0.225899458,
                -0.308674961,
                -0.0115124583,
                0.0730511621,
                -0.495835781,
                0.625569344,
                -0.0438038409,
                0.263472944,
                -0.58485353,
                -0.307070434,
                0.0414443575,
                -0.321789205,
                0.966559,
            ],
            "month_cos": -1.0,
        }
    ]

# Test ranking deployment
ranked_candidates = ranking_deployment.predict(inputs=test_ranking_input)

# Retrieve article ids of the top recommended items
recommendations = get_top_recommendations(ranked_candidates["predictions"], k=3)
print(recommendations)

Check logs in case of failure:

In [ ]:
ranking_deployment.get_logs(component="predictor", tail=200)

# Deploying the query inference pipeline

In [ ]:
query_model_deployment = (
    hopsworks_integration.two_tower_serving.HopsworksQueryModel.deploy()
)

At this point, you have registered your deployment. To start it up you need to run:

In [ ]:
query_model_deployment.start()

## <span style="color:#ff5f27"> Testing the inference pipeline </span>

Define a test input example:

In [ ]:
data = [
    {
        "customer_id": "d327d0ad9e30085a436933dfbb7f77cf42e38447993a078ed35d93e3fd350ecf",
        "transaction_date": "2022-11-15T12:16:25.330916",
    }
]

Test out the deployment:

In [ ]:
ranked_candidates = query_model_deployment.predict(inputs=data)

# Retrieve article ids of the top recommended items
recommendations = get_top_recommendations(ranked_candidates["predictions"], k=3)
print(recommendations)

# <span style="color:#ff5f27"> Stopping the Hopsworks deployment </span>

Stop the deployment when you're not using it.

In [ ]:
ranking_deployment.stop()
query_model_deployment.stop()

## <span style="color:#ff5f27"> Inspecting the deployments in Hopsworks UI </span>

Go to [Hopsworks UI](https://www.hopsworks.ai/), **Data Science → Deployments** section and inspect the newly created deployments.

---

In [ ]:
notebook_end_time = time.time()
notebook_execution_time = notebook_end_time - notebook_start_time

logger.info(
    f"⌛️ Notebook Execution time: {notebook_execution_time:.2f} seconds ~ {notebook_execution_time / 60:.2f} minutes"
)

# <span style="color:#ff5f27">→ Next Steps </span>

The last step is to schedule the materialization jobs.